In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


SVM

In [ ]:
!pip install scikit-learn
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam # Add this line to import the Adam
# Split data into training and test sets
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import mean_absolute_error # Import the mean_absolute_error function
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline


%load_ext tensorboard

import os
log_dir = os.path.join("logs") # Puedes cambiar "logs" por el nombre que quieras
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Cargar los datos (suponiendo que están en un archivo CSV)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM.csv')

#ARREGLO PARA CALC_IMC NULL
data['CALC_IMC'] = data.groupby('ID')['CALC_IMC'].transform(lambda x: round(x.fillna(x.mean()), 0))
#eliminar filas con datos nulos en IMC
data = data.dropna(subset=['CALC_IMC'])

# Contar registros por ID, ignorando los NaN
#conteo_registros_por_persona = data.groupby('ID')['ID'].count()

# Personas con exactamente 4 registros (sin considerar NaN)
#personas_con_4_registros = conteo_registros_por_persona[conteo_registros_por_persona == 4]
#print(personas_con_4_registros)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv', index=False)
data = pd.read_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_1.csv')

# Normalize numerical variables
columns_to_scale = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION','CIGARRETTES','DRINK','CALC_IMC', 'AGE']
scaler = MinMaxScaler(feature_range=(0, 1))
#scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
#data[columns_to_scale] = scaled_data
data.loc[:, columns_to_scale] = scaled_data

# Convert categorical variables to one-hot encoding
# Crear un objeto OneHotEncoder
columns_to_transform = ['SEX','ALONE','PHYSICAL_ACTIVITY']
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(data[columns_to_transform])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_transform))
data = pd.concat([data, encoded_df], axis=1)

#data = data.drop(columns=['SEX','ALONE','PHYSICAL_ACTIVITY'])

# Agrupar por ID y ordenar por 'Evaluación'
#data_grouped = data.sort_values('evaluacion').groupby('ID')

#data = data.drop(columns=['ID','año','evaluación'])

#data.head()
#print(data.columns)

data.to_csv('/content/drive/My Drive/PREDICTIVE MODEL IC/DATA/DATOS_OBSERVACION_VAR_NOTNULL2010-2022_TRANSFORM_ENCODED.csv', index=False)

# Definimos las columnas de las variables a predecir (target)
target_cols = ['LOCOMOTION','SENSORY','VITALITY','PSICHOLOGICAL','COGNITION']

# Prepare sequences and targets
def create_dataset(data, look_back=3):
  sequences = []
  targets = []
  locomotions = []
  sensories = []
  vitalities = []
  psichologicals = []
  cognitions = []
  auxiliares = []
  for person_id in data['ID'].unique():
    person_data = data[data['ID'] == person_id].sort_values('evaluacion')

     # Get the indices of the relevant columns
    relevant_columns = ['LOCOMOTION', 'SENSORY', 'VITALITY','PSICHOLOGICAL', 'COGNITION',
                        'ALONE_PARTNERED', 'ALONE_UNCOUPLED',
                        'PHYSICAL_ACTIVITY_SEDENTARY', 'PHYSICAL_ACTIVITY_MILD','PHYSICAL_ACTIVITY_MODERATE', 'PHYSICAL_ACTIVITY_VIGOROUS',
                        'CIGARRETTES','DRINK', 'CALC_IMC']
    auxiliar_columns = ['AGE','SEX_FEMALE', 'SEX_MALE']
     # Select the relevant columns first
    person_data_values = person_data[relevant_columns].values
    person_data_auxiliar = person_data[auxiliar_columns].values

    # Create sequences and targets
    if len(person_data_values) < look_back + 1:
        continue
    sequence = person_data_values[:look_back, :]
    target = person_data_values[look_back, data.columns.get_indexer(target_cols)]
    locomotion = person_data_values[look_back, 0]
    sensory = person_data_values[look_back, 1]
    vitality = person_data_values[look_back, 2]
    psichological = person_data_values[look_back, 3]
    cognition = person_data_values[look_back, 4]
    auxiliar = person_data_auxiliar[look_back, :]
    sequences.append(sequence)
    targets.append(target)
    locomotions.append(locomotion)
    sensories.append(sensory)
    vitalities.append(vitality)
    psichologicals.append(psichological)
    cognitions.append(cognition)
    auxiliares.append(auxiliar)
  return np.array(sequences), np.array(targets), np.array(locomotions), np.array(sensories), np.array(vitalities), np.array(psichologicals), np.array(cognitions), np.array(auxiliares)

# Create sequences and targets
look_back = 3
sequences, targets, locomotions, sensories, vitalities, psichologicals, cognitions, auxiliares = create_dataset(data, look_back)

pd.DataFrame(sequences[0]).head()
sequences.shape
sequences.shape[2]

sequences.shape[2]
#pd.DataFrame(targets[0]).head()

auxiliares.shape
auxiliares.shape[1]

targets.shape

# TRAINNING AND TEST

# Assuming 'sequences' and 'auxiliares' are your input features,
# and 'locomotions', 'sensories', etc. are your targets
# Split into 80% training and 20% testing
(
    sequences_train,
    sequences_test,
    auxiliares_train,
    auxiliares_test,
    locomotions_train,
    locomotions_test,
    sensories_train,
    sensories_test,
    vitalities_train,
    vitalities_test,
    psichologicals_train,
    psichologicals_test,
    cognitions_train,
    cognitions_test,
) = train_test_split(
    sequences,
    auxiliares,
    locomotions,
    sensories,
    vitalities,
    psichologicals,
    cognitions,
    test_size=0.2,
    random_state=42,  # Optional: for reproducibility
)


# Define input shapes
sequential_input_shape = (sequences.shape[1], sequences.shape[2])  # Adjust num_sequential_features
auxiliary_input_shape = (auxiliares.shape[1],)  # Adjust num_auxiliary_features

# Create the model
def create_model_svm():
    # Flatten the sequential data for SVM
    sequences_train_flattened = sequences_train.reshape(sequences_train.shape[0], -1)
    sequences_test_flattened = sequences_test.reshape(sequences_test.shape[0], -1)

    # Create a pipeline with StandardScaler and MultiOutputRegressor for SVM
    model = make_pipeline(StandardScaler(),
                         MultiOutputRegressor(SVR(kernel='rbf')))
    # Combine sequential and auxiliary data for training
    X_train = np.concatenate([sequences_train_flattened, auxiliares_train], axis=1)

    # Combine target variables for training
    y_train = np.column_stack([locomotions_train, sensories_train, vitalities_train,
                               psichologicals_train, cognitions_train])

    # Fit the SVM model
    model.fit(X_train, y_train)

    # Combine sequential and auxiliary data for testing (for consistency)
    X_test_final = np.concatenate([sequences_test_flattened, auxiliares_test], axis=1)

    # Combine target variables for testing (for consistency)
    y_test_final = np.column_stack([locomotions_test, sensories_test, vitalities_test,
                                  psichologicals_test, cognitions_test])

    return model, X_test_final, y_test_final


# Función para calcular MAE con intervalo de confianza usando Bootstrap (CÓDIGO NUEVO)
def calcular_intervalos_confianza(model, X_test, y_test, num_bootstrap_samples=1000):
    bootstrap_maes = []
    for _ in range(num_bootstrap_samples):
        X_test_resampled, y_test_resampled = resample(X_test, y_test, replace=True)
        predictions = model.predict(X_test_resampled)

     # Calculate MAE for each target and store them
        maes_for_this_bootstrap = [mean_absolute_error(y_test_resampled[:, i], predictions[:, i])
                                 for i in range(y_test.shape[1])]
        bootstrap_maes.append(maes_for_this_bootstrap)

    bootstrap_maes = np.array(bootstrap_maes)
    lower_percentile = 2.5  # For 95% CI
    upper_percentile = 97.5  # For 95% CI

    # Calculate CI directly for MAE values
    confidence_intervals = []
    for i in range(y_test.shape[1]):
        lower_bound = np.percentile(bootstrap_maes[:, i], lower_percentile)
        upper_bound = np.percentile(bootstrap_maes[:, i], upper_percentile)
        confidence_intervals.append((lower_bound, upper_bound))

    return confidence_intervals


# Create and train the model
model, X_test, y_test = create_model_svm()

# Define target names here
target_names = ['LOCOMOTION', 'SENSORY', 'VITALITY', 'PSICHOLOGICAL', 'COGNITION']

# --- Evaluation before Confidence Intervals ---
predictions = model.predict(X_test)  # Make predictions on the original test data

# --- Calculate Total Loss ---
total_loss = 0
for i in range(len(target_names)):
    loss_for_target = mean_absolute_error(y_test[:, i], predictions[:, i])
    total_loss += loss_for_target

total_loss /= len(target_names)  # Average the losses

print(f"Total Loss: {total_loss:.4f}")


# Print evaluation metrics (e.g., MAE)
target_names = ['LOCOMOTION', 'SENSORY', 'VITALITY', 'PSICHOLOGICAL', 'COGNITION']
for i in range(len(target_names)):
    mae = mean_absolute_error(y_test[:, i], predictions[:, i])
    print(f"MAE for {target_names[i]}: {mae:.4f}")



# Calculate confidence intervals
confidence_intervals = calcular_intervalos_confianza(model, X_test, y_test)

# Make predictions using the trained SVM model
predictions = model.predict(X_test)

# Evaluate performance (including confidence intervals)
for i in range(len(target_names)):
    mae = mean_absolute_error(y_test[:, i], predictions[:, i])
    lower_bound, upper_bound = confidence_intervals[i]
    print(f"MAE for {target_names[i]}: {mae:.4f} (95% CI: [{lower_bound:.4f}, {upper_bound:.4f}])")


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<ipython-input-6-cabf4bd70dd4>:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.         0.81818182 0.90909091 ... 0.72727273 0.54545455 0.81818182]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data
<ipython-input-6-cabf4bd70dd4>:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.61428571 0.65714286 0.71428571 ... 0.32857143 0.38571429 0.44285714]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[:, columns_to_scale] = scaled_data


Total Loss: 0.1183
MAE for LOCOMOTION: 0.1599
MAE for SENSORY: 0.1051
MAE for VITALITY: 0.1239
MAE for PSICHOLOGICAL: 0.1169
MAE for COGNITION: 0.0859
MAE for LOCOMOTION: 0.1599 (95% CI: [0.1483, 0.1720])
MAE for SENSORY: 0.1051 (95% CI: [0.0978, 0.1127])
MAE for VITALITY: 0.1239 (95% CI: [0.1163, 0.1313])
MAE for PSICHOLOGICAL: 0.1169 (95% CI: [0.1071, 0.1282])
MAE for COGNITION: 0.0859 (95% CI: [0.0796, 0.0923])
